In [2]:
import time
import sys
import numpy as np


class Context_Queue(object):
    """
         功能：智能客服语义分析引擎-上下文存储机制队列
         输入：
         输出：
         方法：构建存储上下文的队列
    """

    def __init__(self,
                 size=3,
                 time_interval=0.5  # 小时为单位，超过time_interval时间未操作清除该数据
                 ):
        ###   STEP1：定义初始化数据  ###
        self.size = size
        self.time_interval = time_interval
        self.UserText = {}
        self.UserText_Time = {}

    def is_requesId(self, requesId, userId_TextList):
        # 判断队列中是否是同一个session会话
        for text in userId_TextList:
            if requesId in text:
                return True
            else:
                return False

    def is_content(self, Text, userId_TextList):
        # 判断新传入得text是否与前两个队列数据相同
        if len(userId_TextList) >= 2:
            if Text in userId_TextList[len(userId_TextList) - 1] and Text in userId_TextList[len(userId_TextList) - 2]:
                return True
        else:
            return False

    # 删除超时的队列数据（time_interval单位为时间）
    def delete_usertext(self):
        if self.UserText:
            now_time = time.time()
            Time_np = np.array(list(self.UserText_Time.values()))
            Timeout_queue = Time_np[:, 1] < str(now_time - self.time_interval * 60 * 60)
            result_time = np.column_stack((Time_np, Timeout_queue))
            for ii in result_time:
                if ii[2] == str(True):
                    if ii[0] in self.UserText.keys():
                        self.UserText.pop(ii[0])
                        self.UserText_Time.pop(ii[0])
        # return UserText, UserText_Time
        
    def clear_all(self):
        self.UserText.clear()
        self.UserText_Time.clear()

    def context_queue(self, Text, userId, requesId):
        Text_session = (Text, requesId)
        Time_session = (userId, time.time())
        if userId in self.UserText.keys() and userId in self.UserText_Time.keys():
            if self.is_requesId(requesId, self.UserText[userId]):
                if self.is_content(Text, self.UserText[userId]):
                    print("重复三遍以上，转人工吧。。。")
                    # return "转人工"
            else:
                self.UserText[userId].clear()
        else:
            self.UserText[userId] = []
            self.UserText_Time[userId] = []
        self.UserText[userId].append(Text_session)
        self.UserText_Time[userId] = Time_session
        if self.UserText[userId] is not None and len(self.UserText[userId]) <= self.size:
            pass
        else:
            self.UserText[userId].remove(self.UserText[userId][0])
        # print("TextList", self.UserText[userId])
        # return UserText, UserText_Time


robot_context_queue = Context_Queue(size=3, time_interval=0.5)

In [3]:
import fastText as fasttext
import jieba
import re
jieba.load_userdict('../sn_ori_data/finWordDict.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 2.209 seconds.
Prefix dict has been built succesfully.


In [4]:
fasttext_model=fasttext.load_model('./sn_cate/classf_knowledge.model')

In [5]:
stopwords = [line.strip() for line in open('../sn_ori_data/stop_words.txt', 'r', encoding='utf-8').readlines()]
stopwords.append(' ')
stopwords.append('\n')
stopwords.append('\\n')

In [6]:
def jieba_cut(text):
        """
            功能：调用结巴分词函数,去停用词
            输入：标准问题
            输出：问题分词后的List
        """
        text_words = jieba.cut(text, cut_all=False)
        text_list = [word for word in text_words if word not in stopwords]
        return text_list

def outstr_str(text):
    outstr=''
    for word in jieba_cut(text):
        if word != '\t':
            outstr += word
            outstr += " "
    return outstr

In [7]:
def fasttext_classification_single(text):
    outstr = ''
    for word in jieba_cut(text):
        if word != '\t':
            outstr += word
            outstr += " "
    cate=fasttext_model.predict(outstr,k=2)
    return cate

In [8]:
cate1=fasttext_classification_single('你好')

In [460]:
cate1

(('__label__易付宝账户', '__label__理财'), array([0.92601335, 0.06774949]))

In [9]:
def fasttext_classification_context(Text,userId,requesId,threshold=0.8):
    robot_context_queue.context_queue(Text, userId, requesId)
    print("robot_context_queue.UserText",robot_context_queue.UserText)
    len_userId_UserText=len(robot_context_queue.UserText[userId])
    if len_userId_UserText==1:
        text_0=robot_context_queue.UserText[userId][len_userId_UserText-1][0]       #当前输入值
        outstr_0 = outstr_str(text_0)
        cate_0=fasttext_model.predict(outstr_0,k=2)
        print("outstr_0",outstr_0)
        print("cate_0",cate_0)
        cate = re.sub('__label__','',cate_0[0][0])
        return cate
    elif len_userId_UserText==2:
        text_0=robot_context_queue.UserText[userId][len_userId_UserText-1][0]       #当前输入值
        text_1=robot_context_queue.UserText[userId][len_userId_UserText-2][0]      #前一条数据
        outstr_0 = outstr_str(text_0)
        cate_0=fasttext_model.predict(outstr_0,k=2)
        print("outstr_0",outstr_0)
        print("cate_0",cate_0)
        if cate_0[1][0]>=threshold:
            cate = re.sub('__label__','',cate_0[0][0])
            return cate
        else:
            text_01=text_1+text_0
            outstr_01 = outstr_str(text_01)
            cate_01=fasttext_model.predict(outstr_01,k=2)
            print("outstr_01",outstr_01)
            print("cate_01",cate_01)
            cate=re.sub('__label__','',cate_01[0][0])
            return cate
    elif len_userId_UserText==3:
        text_0=robot_context_queue.UserText[userId][len_userId_UserText-1][0]       #当前输入值
        text_1=robot_context_queue.UserText[userId][len_userId_UserText-2][0]      #前一条数据
        text_2=robot_context_queue.UserText[userId][len_userId_UserText-3][0]      #前前一条数据
        outstr_0 = outstr_str(text_0)
        cate_0=fasttext_model.predict(outstr_0,k=2)
        print("outstr_0",outstr_0)
        print("cate_0",cate_0)
        if cate_0[1][0]>=threshold:
            cate = re.sub('__label__','',cate_0[0][0])
            return cate
        else:
            text_01=text_1+text_0
            outstr_01 = outstr_str(text_01)
            cate_01=fasttext_model.predict(outstr_01,k=2)
            print("outstr_01",outstr_01)
            print("cate_01",cate_01)
            if cate_01[1][0]>=threshold:
                cate=re.sub('__label__','',cate_01[0][0])
                return cate
            else:
                text_012=text_2+text_1+text_0
                outstr_012 = outstr_str(text_012)
                cate_012=fasttext_model.predict(outstr_012,k=2)
                print("outstr_012",outstr_012)
                print("cate_012",cate_012)
                cate=re.sub('__label__','',cate_012[0][0])
                return cate
    else:
        cate=fasttext_classification_single(Text)
        return cate

In [10]:
cate=fasttext_classification_context('解释一下吧','1','111')

robot_context_queue.UserText {'1': [('解释一下吧', '111')]}
outstr_0 解释一下 
cate_0 (('__label__易付宝账户', '__label__理财'), array([0.92601335, 0.06774949]))


In [11]:
cate

'易付宝账户'

In [12]:
robot_context_queue.clear_all()

In [13]:
robot_context_queue.UserText

{}